In [1]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [76]:
# test_data = pd.read_csv('../DATA/test_normalized_4labels.csv', index_col=0)
# training_data = pd.read_csv(
#     '../DATA/train_normalized_4labels.csv', index_col=0)

test_data = pd.read_csv('../DATA/test_downsampling.csv', index_col=0)
training_data = pd.read_csv(
    '../DATA/training_downsampling.csv', index_col=0)


In [77]:
np.unique(test_data['CASE_STATUS'])

array(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'DENIED', 'WITHDRAWN'],
      dtype=object)

In [78]:
labels = {'CERTIFIED': 0, 'CERTIFIED-WITHDRAWN': 1, 'DENIED': 2, 'WITHDRAWN':3}
test_data['CASE_STATUS'] = test_data['CASE_STATUS'].apply(lambda x: labels[x])
training_data['CASE_STATUS'] = training_data['CASE_STATUS'].apply(lambda x: labels[x])

In [79]:
train_data, val_data = train_test_split(training_data, test_size=0.2)

In [80]:
train_labels = np.array(train_data.pop('CASE_STATUS'))
val_labels = np.array(val_data.pop('CASE_STATUS'))
test_labels = np.array(test_data.pop('CASE_STATUS'))

train_data.pop('EMPLOYER_NAME')
test_data.pop('EMPLOYER_NAME')
val_data.pop('EMPLOYER_NAME')

train_features = np.array(train_data)
val_features = np.array(val_data)
test_features = np.array(test_data)



In [81]:
transfer_dict = {0: [1,0,0,0],  1:[0,1,0,0], 2: [0,0,1,0], 3:[0,0,0,1]}
train_labels = np.array([transfer_dict[i] for i in train_labels] )
test_labels = np.array([transfer_dict[i] for i in test_labels] )
val_labels = np.array([transfer_dict[i] for i in val_labels])


In [82]:
print(train_features.shape)
print(test_features.shape)
print(val_features.shape)
print(train_labels.shape)
print(test_labels.shape)
print(val_labels.shape)


(79193, 420)
(730531, 420)
(19799, 420)
(79193, 4)
(730531, 4)
(19799, 4)


In [114]:
METRICS = [
    keras.metrics.mean_squared_error,
    keras.metrics.mean_absolute_error,
    keras.metrics.mean_absolute_percentage_error,
    keras.metrics.categorical_accuracy,
]


def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          256, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dense(
          64, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4, activation='softmax',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.CategoricalHinge(),
      metrics=metrics)

  return model


In [117]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)
model = make_model()
model.summary()


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 256)               107776    
_________________________________________________________________
dense_43 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 4)                 260       
Total params: 124,484
Trainable params: 124,484
Non-trainable params: 0
_________________________________________________________________


In [118]:
model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(val_features, val_labels),
    callbacks=[early_stopping],
    verbose=1)


Epoch 1/100
39/39 [==============================] - 1s 15ms/step - loss: 0.8140 - mean_squared_error: 0.1344 - mean_absolute_error: 0.2679 - mean_absolute_percentage_error: 133931992.0000 - categorical_accuracy: 0.5893 - val_loss: 0.6415 - val_mean_squared_error: 0.1106 - val_mean_absolute_error: 0.1895 - val_mean_absolute_percentage_error: 94739120.0000 - val_categorical_accuracy: 0.6878
Epoch 2/100
39/39 [==============================] - 0s 10ms/step - loss: 0.6149 - mean_squared_error: 0.1125 - mean_absolute_error: 0.1751 - mean_absolute_percentage_error: 87573848.0000 - categorical_accuracy: 0.6971 - val_loss: 0.5744 - val_mean_squared_error: 0.1103 - val_mean_absolute_error: 0.1607 - val_mean_absolute_percentage_error: 80335944.0000 - val_categorical_accuracy: 0.7107
Epoch 3/100
39/39 [==============================] - 0s 10ms/step - loss: 0.5650 - mean_squared_error: 0.1071 - mean_absolute_error: 0.1587 - mean_absolute_percentage_error: 79341600.0000 - categorical_accuracy: 0.7

In [119]:
prediction = model.predict(test_data)

In [120]:
prediction

array([[1.38155569e-03, 2.40176490e-07, 9.98541951e-01, 7.62843483e-05],
       [9.98576999e-01, 2.81030632e-04, 1.13979576e-03, 2.10587609e-06],
       [9.92561638e-01, 7.43769063e-03, 5.86193551e-07, 1.23971134e-07],
       ...,
       [1.00000000e+00, 2.10931326e-08, 1.30545404e-17, 2.65401522e-16],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.93461846e-09, 1.27565313e-14, 1.33639269e-08, 1.00000000e+00]],
      dtype=float32)

In [121]:
np.mean(keras.metrics.categorical_accuracy(test_labels,prediction).numpy())

0.85476863

In [122]:
prediction = np.array([list(i).index(max(i)) for i in prediction])


In [123]:
truelabels = np.array([list(i).index(max(i)) for i in test_labels])


In [124]:
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
f1_score(truelabels, prediction, average=None)


array([0.91555631, 0.90782821, 0.19896587, 0.5147419 ])